In [ ]:
%pylab inline
import couchdb
import pandas as pd
import seaborn as sns

In [ ]:
user = ""
password = ""
couchserver = couchdb.Server("http://%s:%s@localhost:5984/" % (user, password))

In [ ]:
for dbname in couchserver:
    print(dbname)

In [ ]:
db = couchserver['prebidlog']

In [ ]:
rows = db.view('_all_docs', include_docs=True)
data = [row['doc'] for row in rows]
df = pd.DataFrame(data)

In [ ]:
df['dt'] = pd.to_datetime(df['timestamp'], unit='ms')

In [ ]:
auctionId = 'ff04d1dd-6321-4e31-8024-23ea415db605'
df[df['auctionId'] == auctionId]

# Bidder stats

In [ ]:
df.groupby('bidder')['auctionId'].count().sort_values(ascending=False)

In [ ]:
df.groupby('bidder')['auctionId'].count().sort_values(ascending=False).plot(kind='bar')

In [ ]:
bids = df.groupby('bidder')['auctionId'].count()
bidder_freq = bids / df.groupby('bidder')['auctionId'].count().sum() * 100
bidder_freq.sort_values(ascending=False).plot(kind='bar', title="Bid frequency (% bids)")

## Auction density

In [ ]:
auction_bidders = df.groupby('auctionId')['bidder'].count()
auction_bidders

In [ ]:
auction_bidders.describe()

In [ ]:
sns.boxplot(x=auction_bidders)

## Co-occurrence of bidders

In [ ]:
df[['auctionId', 'bidder']]
bag_of_bidders = df.groupby('auctionId')['bidder'].agg(size= len, bidders= lambda x: set(x))
bag_of_bidders

In [ ]:
df.groupby('bidder')['auctionId'].count().sort_values(ascending=False)[:20]

In [ ]:
k = 20
top_k_bidders = df.groupby('bidder')['auctionId'].count().sort_values(ascending=False)[:k-1]
top_k_bidders = list(top_k_bidders.index.values)
top_k_bidders

In [ ]:
bidders = df[df.bidder.isin(top_k_bidders)]

In [ ]:
from itertools import combinations

In [ ]:
edges = bidders.groupby(['auctionId'], group_keys=False).apply(
    lambda x: pd.DataFrame(list(combinations(x['bidder'], 2)), 
                           columns=['bidder1', 'bidder2'])).reset_index(drop=True)
print(edges)


In [ ]:
cooc = pd.crosstab(edges.bidder1, edges.bidder2)
cooc

In [ ]:
sns.heatmap(cooc, robust=True)

# Winner stats

In [ ]:
df['winnerName'] = df[~df['winner'].isnull()]['winner'].apply(lambda x: x['bidder'])
df['winnerBidCpm'] = df[~df['winner'].isnull()]['winner'].apply(lambda x: float(x['bidCpm']) if x['bidCpm'] else 0.0)

Auctions won

In [ ]:
winner = df[~df['winner'].isnull()].groupby('winnerName').count()['auctionId'].sort_values(ascending=False)

Winners CPM

In [ ]:
df[~df['winner'].isnull()].groupby('winnerName').agg({'winnerBidCpm': sum})['winnerBidCpm'].sort_values(ascending=False)

Win rate

In [ ]:
winners = df[~df['winner'].isnull()].groupby('winnerName').count()['winner'].reset_index()
bidders = df.groupby('bidder').count()['auctionId'].reset_index()
bidders['bidder'] = bidders['bidder'].apply(lambda x: x.lower())
merged = pd.merge(winners, bidders, left_on='winnerName', right_on='bidder')
merged['winRate'] = merged['winner'] / merged['auctionId']


In [ ]:
merged[['bidder', 'winRate']].sort_values(by='winRate', ascending=False) * 100